In [6]:
# import torch
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from transformers import AutoTokenizer, AutoModelForSequenceClassification
# from torch.utils.data import DataLoader, Dataset
# from torch.optim import AdamW
# from torch.nn import functional as F

In [7]:
# # Load your dataset (use a sample if RAM is limited)
# df = pd.read_parquet("/content/train-00000-of-00001.parquet")

# # Optional: Use 10% of the data for faster training
# df = df.sample(frac=0.1, random_state=42).reset_index(drop=True)

# # Split dataset into training and validation
# train_texts, val_texts, train_labels, val_labels = train_test_split(
#     df["text"].tolist(), df["label"].tolist(), test_size=0.1, stratify=df["label"]
# )

# # Load MiniLM tokenizer
# tokenizer = AutoTokenizer.from_pretrained("nreimers/MiniLM-L6-H384-uncased")

# # Tokenize the text data
# train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=64)
# val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=64)

# # Convert tokenized data to PyTorch Dataset format
# class EmotionDataset(Dataset):
#     def __init__(self, encodings, labels):
#         self.encodings = encodings
#         self.labels = labels

#     def __getitem__(self, idx):
#         item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
#         item["labels"] = torch.tensor(self.labels[idx])
#         return item

#     def __len__(self):
#         return len(self.labels)

# train_dataset = EmotionDataset(train_encodings, train_labels)
# val_dataset = EmotionDataset(val_encodings, val_labels)

# # Create DataLoaders for batching
# train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=16)

# # Load MiniLM model for sequence classification
# model = AutoModelForSequenceClassification.from_pretrained("nreimers/MiniLM-L6-H384-uncased", num_labels=6)

# # Use AdamW optimizer and set a learning rate
# optimizer = AdamW(model.parameters(), lr=5e-5)

# # Training loop
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

# # Number of epochs
# epochs = 3

# for epoch in range(epochs):
#     model.train()
#     total_loss = 0
#     correct_predictions = 0
#     total_samples = 0

#     for batch in train_loader:
#         optimizer.zero_grad()

#         # Move tensors to the same device as model
#         input_ids = batch["input_ids"].to(device)
#         attention_mask = batch["attention_mask"].to(device)
#         labels = batch["labels"].to(device)

#         # Forward pass
#         outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#         loss = outputs.loss
#         logits = outputs.logits

#         # Backward pass
#         loss.backward()
#         optimizer.step()

#         total_loss += loss.item()

#         # Calculate accuracy
#         _, predicted = torch.max(logits, dim=1)
#         correct_predictions += (predicted == labels).sum().item()
#         total_samples += labels.size(0)

#     avg_loss = total_loss / len(train_loader)
#     accuracy = correct_predictions / total_samples * 100
#     print(f"Epoch {epoch + 1}/{epochs} - Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")

#     # Validation
#     model.eval()
#     val_loss = 0
#     correct_predictions = 0
#     total_samples = 0

#     with torch.no_grad():
#         for batch in val_loader:
#             # Move tensors to the same device as model
#             input_ids = batch["input_ids"].to(device)
#             attention_mask = batch["attention_mask"].to(device)
#             labels = batch["labels"].to(device)

#             # Forward pass
#             outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#             loss = outputs.loss
#             logits = outputs.logits

#             val_loss += loss.item()

#             # Calculate accuracy
#             _, predicted = torch.max(logits, dim=1)
#             correct_predictions += (predicted == labels).sum().item()
#             total_samples += labels.size(0)

#     avg_val_loss = val_loss / len(val_loader)
#     val_accuracy = correct_predictions / total_samples * 100
#     print(f"Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

In [8]:
# model.save_pretrained("emotion_model")
# tokenizer.save_pretrained("emotion_model")

In [9]:
# import shutil

# # Create a zip file from the 'emotion_model' directory
# shutil.make_archive("emotion_model", 'zip', "emotion_model")

In [10]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

# Load model and tokenizer from saved directory
model = AutoModelForSequenceClassification.from_pretrained("/content/text_ED_model_MiniLM-L6-H384")
tokenizer = AutoTokenizer.from_pretrained("/content/text_ED_model_MiniLM-L6-H384")

# Set model to evaluation mode
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 384, padding_idx=0)
      (position_embeddings): Embedding(512, 384)
      (token_type_embeddings): Embedding(2, 384)
      (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-5): 6 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=384, out_features=384, bias=True)
              (key): Linear(in_features=384, out_features=384, bias=True)
              (value): Linear(in_features=384, out_features=384, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=384, out_features=384, bias=True)
              (LayerNorm): LayerNorm((384,), eps=1e-1

In [11]:
id2label = {
    0: "sadness",
    1: "joy",
    2: "love",
    3: "anger",
    4: "fear",
    5: "surprise"
}

In [12]:
def predict_emotion(text):
    # Tokenize input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=64)

    # Forward pass
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # Apply softmax to get probabilities
    probs = F.softmax(logits, dim=1)
    predicted_class = torch.argmax(probs, dim=1).item()

    return id2label[predicted_class], probs[0][predicted_class].item()

In [13]:
text = "I’m really happy with the results!"
label, confidence = predict_emotion(text)
print(f"Predicted Emotion: {label} (Confidence: {confidence:.2f})")

Predicted Emotion: joy (Confidence: 0.94)
